In [7]:
from bs4 import BeautifulSoup
from requests import get, post
import urllib.parse as up
import urllib

from threading import Thread

from time import sleep

def encode_and_replace(li):
    li.encode('UTF-8')
    return li.attrs['href'] if 'href' in li.attrs else ''

class page(object):
    def getHiperlinks(self):
        query='banana'
        query_template = 'https://uk.wikipedia.org?{}'
        url = query_template.format(up.urlencode({'q': query}))
        if self.title =='https://uk.wikipedia.org':
            url = 'https://uk.wikipedia.org'
        else: url = 'https://uk.wikipedia.org' + self.title
        
        req = get(url)
        soup = BeautifulSoup(req.content.decode(req.encoding), "lxml")
        
        arList = [encode_and_replace(li) for li in soup.findAll('a')]
        arList = list(set([i for i in arList if self.validation(i)]))
        return arList
    
    def __init__(self, title):
        self.title=title
        self.pageList=self.getHiperlinks()
    
    def __str__(self):
        s = "(" + str(self.title) + ":\n  "
        for i in self.pageList:
            s += str(i) + ";  "
        s += ')'
        return s

    def validation(self, ar):
        bad = min(list(map(lambda x: not x in ar, features_not)))
        return bad

features_not = ['Special', '.wikipedia.org', 
                'wikidata', 'wiktionary', 'wikibooks', 
                'wikiquote', 'en.wiki',
                'wikinews', 'meta.w', 'mediawiki', '/w/']


In [8]:
from time import time
from copy import deepcopy
class crawler(object):
    def __init__(self, init, n = 3, batch = 250):
        self.data=[]
        self.unvisitedPage=[]
        self.unvisitedPage.append(init)
        self.n = n
        self.workers = 0
        self.newSet = []
        self.batch = batch
        
    def __contains__(self, value):
        result = [i.title == value for i in self.data]
        return max(result) if result else False
        
    def add(self, page):
        self.data.append(page)
        
    def __str__(self):
        s = "Data:\n"
        for i in self.data:
            s += str(i) + "\n"
        s += "unvisited pages:\n"
        for i in self.unvisitedPage:
            s += str(i) + "\n"
        return s
    
    def one_hand(self, p):
        page_i = page(p)
        if not p in self:
            self.add(page_i)
        self.newSet += [i for i in page_i.pageList if 'http' not in i and 'irc' not in i] 
        self.workers -= 1
    
    def toCraw(self):
        self.newSet = []
        for i in self.unvisitedPage[:self.batch]:
            while not self.has_free_hands():
                sleep(0.2)
            self.workers += 1
            Thread(target = self.one_hand, args = (i,)).start()
        b = time()
        #print('workers', self.workers, ' ')
        while(self.workers > 0):
            sleep(0.2)
        #print('waiting:', time() - b)
        
        self.x = 0
        self.newSet = list(set(self.newSet) - set([i.title for i in self.data]))
        self.unvisitedPage = self.unvisitedPage[self.batch:] + self.newSet
        self.newSet = []

    
    def has_step(self):
        return len(self.unvisitedPage) > 0
        
        
    def has_free_hands(self):
        return self.n > self.workers
    
    def filter_url_img(self, uns = False):
        img = ['.img', '.jpg', '.bmp', '.png', '.svg']
        res = []
        if not uns:
            for i in self.data:
                if  max(map(lambda x: x in i.title, img)) and '/wiki/' not in i.title:
                    res.append(i.title)
            return res
        else:
            return self.unvisitedPage
    
    def download_one_img(self, i, img):
        try:
            with urllib.request.urlopen('https:' + img) as url:
                s = url.read()
            t = img.split('.')[-1]
            out = open('images/' + str(i) + '.' + t, 'wb')
            out.write(s)
            out.close()
            self.workers -= 1
        except:
            self.workers -= 1
            
    def download_img(self, uns=False):
        res = self.filter_url_img(uns = uns)
        for i, img in enumerate(res):
            while(self.workers >= self.n):
                sleep(0.2)
            self.workers += 1
            Thread(target = self.download_one_img, args = (i, img)).start()
        print('uploading done!')

In [9]:
for i in [16]:
    total = time()
    moWiki = 'https://uk.wikipedia.org'
    cr = crawler(moWiki, n = i)
    print(cr)
    while(cr.has_step()):
        b = time()
        cr.toCraw()
        print('visited', len(cr.data), 'unvisited', len(cr.unvisitedPage))
    print('threads: ', i, 'time: ',time() - total)


Data:
unvisited pages:
https://uk.wikipedia.org

visited 1 unvisited 390
visited 251 unvisited 32111
visited 501 unvisited 71572
visited 751 unvisited 93214
visited 999 unvisited 122335
visited 1249 unvisited 146620
visited 1498 unvisited 173835
visited 1747 unvisited 198380
visited 1991 unvisited 224969
visited 2241 unvisited 250961
visited 2490 unvisited 273666
visited 2740 unvisited 295890
visited 2989 unvisited 320898
visited 3239 unvisited 347075
visited 3486 unvisited 373838
visited 3736 unvisited 397703
visited 3985 unvisited 421988
visited 4235 unvisited 445127
visited 4485 unvisited 467170
visited 4735 unvisited 489142
visited 4985 unvisited 512998
visited 5235 unvisited 536106
visited 5480 unvisited 558657
visited 5730 unvisited 582979
visited 5977 unvisited 608650
visited 6227 unvisited 629687
visited 6475 unvisited 655063
visited 6725 unvisited 677543
visited 6970 unvisited 701351
visited 7218 unvisited 726271
visited 7466 unvisited 749701
visited 7713 unvisited 774456
visi

Exception in thread Thread-71514:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\requests\packages\urllib3\connection.py", line 138, in _new_conn
    (self.host, self.port), self.timeout, **extra_kw)
  File "C:\ProgramData\Anaconda3\lib\site-packages\requests\packages\urllib3\util\connection.py", line 75, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 743, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py", line 594, in urlopen
    chunked=chunked)
  File "C:\ProgramData\Anaconda3\lib\site-packages\requests\packages\urllib3\connectionpool.py", line 350, in _mak

KeyboardInterrupt: 

In [12]:
print(len(cr.filter_url_img()))

92


In [14]:
cr.download_img(uns=True)

KeyboardInterrupt: 

In [ ]:
moWiki = 'https://mo.wikipedia.org'
lis = ['https://mo.wikipedia.org']
for pages in cr.data:
    lis+= pages.pageList
lis = list(set(lis))

In [11]:
index = {}
for i,k in enumerate(lis):
    index.update({k:i})

In [12]:
matrix = []
zero=[]
x=0
for i in range(0,len(cr.data)):
    zero=[]
    for i in range(0,len(cr.data)):
        zero.append(0)
    matrix.append(zero)
for i in cr.data:
    for j in i.pageList:
        matrix[index[i.title]][index[j]]=1

In [13]:
gr = matrix
#print(gr)

In [14]:
dist=[]
for i in range(0,len(gr[0])):
        dist.append(0)
x=0
poss=[240]      
visited=set()
curr=set()
while len(visited)!=len(gr)-1:
    x+=1
    print(len(visited))
    for k in list(poss):
        for i in range(0,len(gr[k])):
            if gr[k][i]==1 and i!=240 and i not in visited:
                #print(i)
                dist[i]=x
                visited.update({i})
                curr.update({i})
                visited.update({k})
    poss=set(curr)
print(dist)

0
11
45
148
348
421
429
[1, 2, 4, 4, 3, 3, 3, 2, 4, 5, 5, 3, 4, 4, 4, 3, 4, 4, 5, 5, 3, 4, 4, 4, 3, 2, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 2, 5, 4, 3, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 3, 3, 4, 4, 4, 4, 2, 5, 4, 4, 4, 3, 4, 3, 3, 5, 1, 3, 5, 5, 5, 4, 5, 3, 3, 4, 6, 5, 3, 4, 4, 4, 3, 5, 4, 4, 4, 5, 3, 2, 4, 4, 6, 2, 4, 3, 4, 3, 4, 5, 4, 4, 4, 3, 4, 4, 5, 5, 3, 5, 3, 3, 3, 3, 7, 1, 2, 3, 5, 5, 5, 5, 2, 3, 4, 2, 4, 5, 4, 4, 5, 4, 4, 5, 4, 3, 7, 3, 5, 4, 2, 3, 5, 3, 3, 4, 5, 4, 4, 4, 2, 5, 3, 4, 3, 4, 4, 4, 3, 4, 5, 5, 2, 4, 4, 2, 3, 4, 3, 3, 3, 4, 4, 3, 5, 0, 4, 6, 2, 5, 3, 4, 4, 4, 3, 1, 3, 4, 3, 4, 3, 3, 5, 4, 5, 3, 5, 5, 3, 4, 4, 4, 3, 2, 4, 4, 4, 3, 3, 4, 2, 4, 4, 4, 4, 4, 3, 4, 3, 4, 5, 4, 2, 3, 3, 4, 2, 4, 4, 5, 4, 3, 4, 4, 0, 3, 3, 4, 4, 4, 3, 4, 3, 5, 6, 2, 3, 5, 3, 3, 6, 2, 3, 2, 3, 3, 3, 3, 4, 2, 5, 4, 5, 1, 4, 6, 3, 1, 2, 3, 6, 1, 4, 5, 4, 4, 5, 2, 4, 3, 4, 1, 5, 4, 5, 4, 4, 5, 4, 2, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 5, 4, 4, 5, 4, 4, 5, 4, 5, 4, 4, 3, 4, 4, 